# 4a. Daten beschaffen

In diesem Notebook beginnen wir die Datenbeschaffung mit dem Einsammeln aller Märchen-URLs aus der deutschsprachigen Wikisource.

## Inhalte
- Wir beginnen auf der Seite [Kategorie Märchen](https://de.wikisource.org/wiki/Kategorie:M%C3%A4rchen). Eine kleine Webcrawler-Funktion erhielt diese URL als Ausgangspunkt.
- Die Funktion `get_fairytale_urls()` erhält die Ausgangs-URL als Eingabe und lädt deren Inhalte mit der Bibliothek [requests](https://docs.python-requests.org/en/master/index.html) herunter.
- Der HTML-Inhalt der Seite wird mit der [beautifulsoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)-Bibliothek in ein `BeautifulSoup`-Objekt transformiert.
- Wir sammeln URLs zu Märchen-Volltexten ein, indem wir das HTML durchgehen.
- Wenn ein "Nächste Seite"-Link vorhanden ist, folgen wir diesem.
- Wir wiederholen diese Schritte, bis kein weiterer "Nächste Seite"-Link vorhanden ist.
- Wir speichern die URLs als TSV-Datei mit der [pandas](https://pandas.pydata.org/pandas-docs/stable/) library) für die Weiterverarbeitung.

### Quellen: Deutsche Märchen in Wikipedia/Wikisource

- Märchen:
    - https://de.wikipedia.org/wiki/M%C3%A4rchen
    - https://de.wikipedia.org/wiki/Liste_von_M%C3%A4rchen
    - https://commons.wikimedia.org/wiki/Category:Fairy_tales?uselang=de
    - https://de.wikisource.org/wiki/M%C3%A4rchen
- Märchen-Volltexte in Wikisource: 
    - https://de.wikisource.org/wiki/Kategorie:M%C3%A4rchen
    - e.g. https://de.wikisource.org/wiki/K%C3%B6nig_Dro%C3%9Felbart_(1812)

In [1]:
# import the libraries needed in this notebook
# 'standard library' imports
import pathlib
import time

# import a single function with 'from' statement
from urllib.parse import urljoin

# third party library imports
import pandas as pd
import requests

# import a single class with 'from' statement
from bs4 import BeautifulSoup

# initialize Path to "data/" directory to store data in
DATA_DIR = pathlib.Path().cwd().parent.joinpath("data")
RAW_DATA_DIR = DATA_DIR.joinpath("raw")
WIKISOURCE_BASE_URL = "https://de.wikisource.org"

In [2]:
def get_fairytale_urls(url):
    """Retrieve all URLs from a given base URL, iterating through 'next page' links."""
    fairytale_urls = set()
    while True:
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        entries = soup.find(id="mw-pages").find(class_="mw-category")
        relative_urls = entries.find_all("a")
        for relative_url in relative_urls:
            href = relative_url.get("href")
            absolute_url = urljoin(WIKISOURCE_BASE_URL, href)
            fairytale_urls.add(absolute_url)
        # find the link to the next page via the text "Nächste Seite"
        next_page = soup.find("a", string="nächste Seite")
        # leave while loop, if there is no more next_page link found.
        if not next_page:
            break
        url = urljoin(WIKISOURCE_BASE_URL, next_page.get("href"))
        # wait one second before sending a new request to wikisource
        time.sleep(1)
    return sorted(list(fairytale_urls))

In [3]:
FAIRYTALES_BASE_URL = "https://de.wikisource.org/wiki/Kategorie:M%C3%A4rchen"
fairytale_urls = get_fairytale_urls(FAIRYTALES_BASE_URL)

https://de.wikisource.org/wiki/Kategorie:M%C3%A4rchen
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=Bremer+Stadtmusikanten+%23Die+%231857%0ADie+Bremer+Stadtmusikanten+%281857%29#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=dumme+Xailun+%23Der%0ADer+dumme+Xailun#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=geizige+Schwiegertochter+%23Die%0ADie+geizige+Schwiegertochter#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=Hasichenbraut+%231850%0AH%C3%A4sichenbraut+%281850%29#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=Knuppel+aus+dem+Sacke+%23Der%0ADer+Kn%C3%BCppel+aus+dem+Sacke#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefrom=Nacht+auf+dem+Schlachtfeld+%23Die%0ADie+Nacht+auf+dem+Schlachtfeld#mw-pages
https://de.wikisource.org/w/index.php?title=Kategorie:M%C3%A4rchen&pagefr

In [4]:
# How many links to fairytales are there?
len(fairytale_urls)

1934

In [5]:
# display the first 10 links to fairytales in wikisource
fairytale_urls[:10]

['https://de.wikisource.org/wiki/%C3%84skenbridel_Sa%C3%BAnsidel',
 'https://de.wikisource.org/wiki/Ahmed_und_Paribanu',
 'https://de.wikisource.org/wiki/Albanesische_M%C3%A4rchen',
 'https://de.wikisource.org/wiki/Allerlei-Rauh_(1812)',
 'https://de.wikisource.org/wiki/Allerlei-Rauh_(1819)',
 'https://de.wikisource.org/wiki/Allerleihaut',
 'https://de.wikisource.org/wiki/Allerleirauh_(1837)',
 'https://de.wikisource.org/wiki/Allerleirauh_(1840)',
 'https://de.wikisource.org/wiki/Allerleirauh_(1843)',
 'https://de.wikisource.org/wiki/Allerleirauh_(1850)']

In [6]:
# convert list to pandas.Series and store it on disk
fairytale_urls = pd.Series(fairytale_urls, name="url")

# store the urls in a TSV file inside the data/ directory
filename = RAW_DATA_DIR.joinpath("wikisource_fairytale_urls.tsv")
fairytale_urls.to_csv(filename, sep="\t", index=False)

In [7]:
import session_info

session_info.show()